# 1. LoRA - BERT 모델을 사용한 PEFT
BERT 모델을 사용하여 PEFT(LoRA) 사용

In [1]:
# 현재 연결된 커널 환경에 강제로 재설치합니다.
%pip install --force-reinstall torch torchvision torchaudio transformers huggingface_hub

  Using cached torch-2.9.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.24.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached torchaudio-2.9.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manyl

In [ ]:
from huggingface_hub import login

# 위젯 대신 login 함수 안에 토큰을 직접 넣어서 로그인합니다.
# "hf_..." 부분에 복사해둔 토큰을 따옴표 안에 넣어주세요.
login(token="hf_...", write_permission=True)

# 만약 모델을 업로드(저장)까지 해야 한다면 아래처럼 옵션을 추가할 수 있습니다.
# login(token="hf_...", write_permission=True)

## BERT 모델 로드

In [3]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# BERT 로드
base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=6
)

base_model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 데이터세트 로드

In [4]:
from datasets import load_dataset

# 데이터세트
dataset = load_dataset("dair-ai/emotion")
dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0}

## 토크나이저 설정

In [5]:
# 토크나이저
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized = dataset.map(preprocess, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_ds = tokenized["train"]
eval_ds  = tokenized["validation"]

train_ds[0]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'labels': tensor(0),
 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

##  PEFT - Lora 설정과 학습

In [6]:
# peft와 evaluate 라이브러리 설치
%pip install peft evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
# 호환성 문제를 해결하기 위해 transformers 버전을 4.45.2로 재설치합니다.
%pip install transformers==4.45.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.25.1 requires trans

In [8]:
from peft import LoraConfig, get_peft_model
import evaluate

# LoRA 구성
lora_config = LoraConfig(
    task_type="SEQ_CLS",  # 시퀀스 분류
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["query", "key", "value", "output.dense"],
)

# LoRA 적용
model = get_peft_model(base_model, lora_config)

# Trainer 구성
training_args = TrainingArguments(
    output_dir="./lora_bert_mrpc",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="steps",
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none",
)

# 평가 메트릭
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8. 학습 시작
trainer.train()

Using the latest cached version of the module from /home/jayden86/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Fri Nov 21 11:26:24 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
/tmp/ipykernel_243916/1542683702.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
20,1.826700,1.771703,0.338000
40,1.741900,1.721143,0.352000
60,1.718600,1.673490,0.352500
80,1.671500,1.630572,0.351500
100,1.640300,1.597500,0.350000
120,1.608100,1.585284,0.345000
140,1.615700,1.579700,0.351000
160,1.616100,1.579118,0.351500
180,1.594400,1.576738,0.359500
200,1.562400,1.575608,0.353000


TrainOutput(global_step=3000, training_loss=1.1467801710764567, metrics={'train_runtime': 2575.1337, 'train_samples_per_second': 18.64, 'train_steps_per_second': 1.165, 'total_flos': 3192948965376000.0, 'train_loss': 1.1467801710764567, 'epoch': 3.0})

## 성능 평가(Evaluate)

In [10]:
eval_results = trainer.evaluate()
print(f"최종 정확도(Accuracy): {eval_results['eval_accuracy']:.4f}")

최종 정확도(Accuracy): 0.6565


## 모델 저장

LoRA는 모델 전체가 아니라 학습된'어댑터'부분만 저정하면 됩니다. 용량이 매우 작어서 효율적입니다.

In [11]:
# 학습된 LoRA 모델 Adapter 저장
save_dir='./bert-emotion-lora'
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f'모델이 {save_dir}에 저장되었습니다.')

모델이 ./bert-emotion-lora에 저장되었습니다.


## 실제 테스트 (Inference)
저장된 모델이 새로운 문장을 보고 감정을 잘 맞추는지 직접 테스트해 봅니다.

In [12]:
# 테스트: 문장을 넣어 감정 에측해보기
import torch

# 테스트 문장
text = 'I feel so happy and excited today!'

# 입력 변환(토큰화)
inputs = tokenizer(text, return_tensors='pt').to(model.device)

# 모델 예측
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()

# 결과 출력 (Emotion 데이터셋 라벨 매핑)
# 0: sadness, 1: joy, 2: love, 3: anger, 4: fear, 5: surprise
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
print(f"입력 문장: {text}")
print(f"예측 감정: {labels[predicted_class_id]}")

입력 문장: I feel so happy and excited today!
예측 감정: joy
